# Hnefatafl - AlphaZero
## CISC-856 W21 Project
## Written by Tindur Sigurdason and Calen Irwin

##Purpose: 

Combine all project components to demonstrate our AlphaZero-based algorithm's ability to learn Hnefatafl through self play

##Instructions:
- Upload this notebook to a cloud environment like Google Colab 
- Click 'Runtime > Run all' and wait for runtime to restart
- Click the first cell under Load Libraries and then click 'Runtime > Run after' in order to execute all of the remaining cells in the notebook

In [1]:
!rm -rf 'gym-hnef'

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!git clone https://github.com/calenirwin/gym-hnef

In [1]:
%pip install -e gym-hnef

Note: you may need to restart the kernel to use updated packages.


ERROR: gym-hnef is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [ ]:
import os

# restarts the runtime after download github repo and install the custom gym env
def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

# Load Libraries

In [2]:
# load necessary libraries
# highlight this cell and click 'Runtime > Run after', once you have install the 
# gym env and restarted the runtime

# external libraries
import gym
import numpy as np
from matplotlib import pyplot as plt

# local libraries
from gym_hnef import hnef_game, hnef_vars
from gym_hnef.envs import hnef_env
import agent, config, funcs, mcts, memory, model

In [30]:
action_size = 6561 # historical action space => 9^4
# action_size = 625   # mini action space => 5^4

# intialize game state
init_state = hnef_game.init_state('historical')
# init_state = hnef_game.init_state('mini')

In [31]:
# instantiate the neural networks
current_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)
best_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)

In [32]:
# instantiate the game agents
current_player = agent.Agent('current_player', current_NN, init_state.shape, action_size)
best_player = agent.Agent('best_player', best_NN, init_state.shape, action_size)

In [33]:
iteration = 1
max_iter = 2
mem = memory.Memory(config.MEMORY_SIZE)
version = 0

while iteration < max_iter:
    print('Iteration number:', iteration)

    # self play
    scores, mem = funcs.play_matches(best_player, best_player, mem=mem, rule_set='historical')
    print(scores)

    mem.clear_stmemory()

    # if the memory has exceeded its capacity, then initiate a tournament to decide the best player
    if len(mem.ltmemory) >= config.MEMORY_SIZE:

        current_player.replay(mem.ltmemory)

        # tournament between best and current agent
        scores, _ = funcs.play_matches(best_player, current_player, rule_set='historical')
        print(scores)

        # if the current player wins more than the given threshold, reassign the best player to the current player
        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            version += 1
            print(version)
            # changes weights to the new best player's weights
            best_NN.model.set_weights(current_NN.model.get_weights())
    else:
        print('Memory size', str(len(mem.ltmemory)))
    iteration += 1

Iteration number: 1
Starting Episode  1 ...
1/1 [==============================] - 0s 195ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 225ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 194ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 201ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 201ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 202ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 213ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 198ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 193ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 202ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 190ms/step
***Repitition

In [34]:
random_player = agent.RandomAgent('random_player', init_state.shape, action_size)
scores, all_end_states = funcs.evaluate_agents(best_player, random_player, num_games=10, rule_set='historical', switch_sides=True)

1/1 [==============================] - 0s 193ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 202ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 201ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 195ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 200ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 195ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 200ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 196ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 207ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 195ms/step
***Repitition condition met in MCTS
1/1 [==============================] - 0s 220ms/step
Game 0 Final State:
    0   1   2   3   4   5   6   7   8

c:\Users\catal\.conda\envs\env\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


1/1 [==============================] - 0s 194ms/step
Game 1 Final State:
    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 |   |   |   | A | A |   |   | A |   |
  -------------------------------------
1 |   |   |   | A |   |   |   |   |   |
  -------------------------------------
2 |   | A |   |   |   | D |   |   |   |
  -------------------------------------
3 |   |   | A |   |   |   | D |   | A |
  -------------------------------------
4 | A |   |   | D |   | D |   |   | A |
  -------------------------------------
5 |   |   |   |   |   | D | D | A | A |
  -------------------------------------
6 |   |   |   |   |   | D |   |   |   |
  -------------------------------------
7 |   | D |   |   |   |   | A |   |   |
  -------------------------------------
8 | A |   |   | A | K |   | A | A |   |
  -------------------------------------
	Turn: Attacker
Defender win
1/1 [==============================] - 0s 188ms/step


KeyboardInterrupt: 